In [ ]:
import pandas as pd
import numpy as np

from github import Github
from _private.GithubCredentials import my_credentials

from harpcrawler.harprepo.device import DeviceRepo, TemplateDeviceRepo
from harpcrawler.harprepo.peripheral import PeripheralRepo, TemplatePeripheralRepo
from harpcrawler.gsheets import HarpSpreadsheet

In [ ]:
## Setup
gh = Github(login_or_token=my_credentials())

device_template = TemplateDeviceRepo(gh.get_repo("bruno-f-cruz/device.template"))
peripheral_template = TemplatePeripheralRepo(gh.get_repo("bruno-f-cruz/peripheral.template"))

harpSpreadsheet = HarpSpreadsheet("GitHarpCrawler")

In [ ]:
harp_organization = gh.get_organization("harp-tech")

## Get all "Device.*" repositories
device_repos = [DeviceRepo(repo, device_template)\
    for repo in harp_organization.get_repos()\
        if (("device." in repo.full_name) and not("template" in repo.full_name))]

device_template.run_diagnosis(repos_to_validate=device_repos)
device_diagnosis = device_template.print_diagnosis()
print(device_diagnosis)

device_diagnosis["Warnings"] = device_diagnosis["Warnings"].apply(lambda x: str(x))
device_diagnosis["FirmwareVersion"] = device_diagnosis["FirmwareVersion"].apply(lambda x: str(x))
device_diagnosis["HardwareVersion"] = device_diagnosis["HardwareVersion"].apply(lambda x: str(x))
device_diagnosis["HarpProtocolVersion"] = device_diagnosis["HarpProtocolVersion"].apply(lambda x: str(x))

harpSpreadsheet.open_spreadsheet()
harpSpreadsheet.update_spreadsheet("devices", device_diagnosis)


In [ ]:
harp_organization = gh.get_organization("harp-tech")

## Get all "Peripherals.*" repositories

peripheral_repos = [PeripheralRepo(repo, peripheral_template)\
    for repo in harp_organization.get_repos()\
        if (("peripheral." in repo.full_name) and not("template" in repo.full_name))]

peripheral_template.run_diagnosis(repos_to_validate=peripheral_repos)
peripherals_diagnosis = peripheral_template.print_diagnosis()
print(peripherals_diagnosis)

peripherals_diagnosis["Warnings"] = peripherals_diagnosis["Warnings"].apply(lambda x: str(x))
peripherals_diagnosis["HardwareVersion"] = peripherals_diagnosis["HardwareVersion"].apply(lambda x: str(x))

harpSpreadsheet.open_spreadsheet()
harpSpreadsheet.update_spreadsheet("peripherals", peripherals_diagnosis)